In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
%autoreload

In [5]:
import os
os.chdir('/home/jaentrouble/adipose_train3')
print(os.getcwd())

/home/jaentrouble/adipose_train3


In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import imageio as io
import json

img_names = os.listdir('data/done')
img = []
img_name_dict = {}
for idx, name in enumerate(img_names):
    img.append(io.imread('data/done/'+name))
    img_name_dict[name] = idx

json_names = os.listdir('data/save')
data = []
for name in json_names:
    with open('data/save/'+name,'r') as j:
        data.extend(json.load(j))
for datum in data :
    datum['image'] = img_name_dict[datum['image']]

In [ ]:
!git pull
%autoreload

In [12]:
from model_trainer import run_training
from encoder_models import *
from box_models import *
from model_lr import *
from multiprocessing import Process

In [ ]:
encoder_f = hr_5_3_0
box_f = dense_4
lr_f = lr_mul_inv
name = 'hr_5_3_0_d4'
epochs = 20
batch_size = 32
steps_per_epoch = len(data)//batch_size
img_size = (800,600)
mixed_float = True

In [20]:
kwargs = dict(
    encoder_f = encoder_f,
    box_f = box_f,
    lr_f = lr_f,
    name = name,
    epochs = epochs,
    steps_per_epoch=steps_per_epoch,
    mixed_float=mixed_float,
    batch_size=batch_size,
    img=img,
    data=data,
    img_size=img_size,
    mixed_float=mixed_float,
    notebook= True,
)

In [21]:
p = Process(target=run_training, kwargs=kwargs)
p.start()
p.join()